In [1]:
import tensorflow as tf

PATCH_SIZE = 7
PATCH_PIXELS = 7*7

BATCH_SIZE = 100

NUM_THREADS = 4

def read(filenames):
    queue = tf.train.string_input_producer(filenames)

    with tf.name_scope('reader'):
        reader = tf.TFRecordReader()
        _, example = reader.read(queue)
    
    with tf.name_scope('features'):
        features = tf.parse_single_example(example, features={
            'us_patch': tf.FixedLenFeature([], tf.string),
            'mr_patch': tf.FixedLenFeature([], tf.string),
        })
    
    with tf.name_scope('features_decode'):
        image = tf.reshape(tf.decode_raw(features['mr_patch'], tf.float64), [PATCH_SIZE, PATCH_SIZE, 1])
        label = tf.reshape(tf.decode_raw(features['us_patch'], tf.float64), [PATCH_SIZE, PATCH_SIZE, 1])
    
    return tf.train.shuffle_batch([image, label],
        batch_size=BATCH_SIZE,
        num_threads=NUM_THREADS,
        capacity=1000 + 3*BATCH_SIZE,
        min_after_dequeue=1000)

In [2]:
image = tf.placeholder(shape=[None, PATCH_SIZE, PATCH_SIZE, 1], dtype=tf.float32)
label = tf.placeholder(shape=[None, PATCH_SIZE, PATCH_SIZE, 1], dtype=tf.float32)

conv1_weight = tf.Variable(tf.zeros(shape=[3, 3, 1, 3]))
conv1_bias = tf.Variable(tf.zeros(shape=[3]))
conv1 = tf.nn.conv2d(image, conv1_weight, strides=[1, 1, 1, 1], padding='SAME') + conv1_bias

conv2_weight = tf.Variable(tf.zeros(shape=[1, 1, 3, 1]))
conv2 = tf.nn.conv2d(conv1, conv2_weight, strides=[1, 1, 1, 1], padding='SAME')

result = conv2

cost = tf.nn.l2_loss(label-result)

train = tf.train.GradientDescentOptimizer(.5).minimize(cost)

In [3]:
pred = tf.equal(tf.argmax(label, 1), tf.argmax(result, 1))
loss = tf.reduce_mean(tf.cast(pred, tf.float32))

In [ ]:
import time

test_files = ['th-30/{:02}.tfrecord'.format(i) for i in range(1, 8)]
train_files = ['th-30/{:02}.tfrecord'.format(i) for i in range(8, 15)]

batch = read(train_files)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    try:
        step = 0
        
        while not coord.should_stop():
            start_time = time.time()
        
            _, accuracy = sess.run([train, loss], feed_dict={
                image: batch[0].eval(),
                label: batch[1].eval(),
            })
            
            duration_time = start_time - time.time()
            
            if step % 100 == 0:        
                print('step {} with accuracy {}'.format(step, accuracy))
            
            step += 1
    except tf.errors.OutOfRangeError:
        print('done with training')
    finally:
        coord.request_stop()
        coord.join(threads)

step 0 with accuracy 0.477142870426178
step 100 with accuracy 0.5271428823471069
step 200 with accuracy 0.3871428668498993
step 300 with accuracy 0.4357142746448517
step 400 with accuracy 0.3042857050895691
step 500 with accuracy 0.34142857789993286
step 600 with accuracy 0.34142857789993286
step 700 with accuracy 0.3871428668498993
step 800 with accuracy 0.2785714268684387
step 900 with accuracy 0.35285714268684387
step 1000 with accuracy 0.35857143998146057
step 1100 with accuracy 0.39714285731315613
step 1200 with accuracy 0.3514285683631897
step 1300 with accuracy 0.3700000047683716
step 1400 with accuracy 0.45428571105003357
step 1500 with accuracy 0.33571428060531616
step 1600 with accuracy 0.4399999976158142
step 1700 with accuracy 0.41285714507102966
step 1800 with accuracy 0.21714285016059875
step 1900 with accuracy 0.39571428298950195
step 2000 with accuracy 0.3100000023841858
step 2100 with accuracy 0.3400000035762787
step 2200 with accuracy 0.40142858028411865
step 2300 wit